In [1]:
] status
using JuMP
using HiGHS
using DataFrames

#CRIAR UM MODELO
mod = Model(HiGHS.Optimizer)

#CRIAR VARIÁVEIS
@variable(mod,x1>=0)
@variable(mod, 0<=x2<=2)

#CRIAR FUNÇÃO OBJETIVO
@objective(mod, Max, 5x1+4x2)

#CRIAR RESTRIÇÕES
@constraint(mod, c1, 1x1+2x2<=6)
@constraint(mod, c2, 6x1+4x2<=24)

print(mod)

#RESOLVER
optimize!(mod)

#APRESENTAR SOLUÇÃO
solution_summary(mod, verbose=true)

#ANÁLISE DE SENSIBILIDADE
report = lp_sensitivity_report(mod);

function variable_report(xi) 
    return (
        name = name(xi),
        lower_bound = has_lower_bound(xi) ? lower_bound(xi) : -Inf,
        value = value(xi),
        upper_bound = has_upper_bound(xi) ? upper_bound(xi) : Inf,
        reduced_cost = reduced_cost(xi),
        obj_coefficient = coefficient(objective_function(mod), xi),
        allowed_decrease = report[xi][1],
        allowed_increase = report[xi][2],
    )
end

variable_df = DataFrames.DataFrame(variable_report(xi) for xi in all_variables(mod))

function constraint_report(c::ConstraintRef) 
    return (
        name = name(c),
        value = value(c),
        rhs = normalized_rhs(c),
        slack = normalized_rhs(c) - value(c),
        shadow_price = shadow_price(c),
        allowed_decrease = report[c][1],
        allowed_increase = report[c][2],
    )
    end

constraint_df = DataFrames.DataFrame(constraint_report(ci) 
    for (F, S) in list_of_constraint_types(mod) 
        for ci in all_constraints(mod, F, S) 
            if F == AffExpr
    )

Running HiGHS 1.7.2 (git hash: 5ce7a2753): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 6e+00]
  Cost   [4e+00, 5e+00]
  Bound  [2e+00, 2e+00]
  RHS    [6e+00, 2e+01]
Presolving model
2 rows, 2 cols, 4 nonzeros  0s
2 rows, 2 cols, 4 nonzeros  0s
Presolve : Reductions: rows 2(-0); columns 2(-0); elements 4(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -2.4999926288e+00 Ph1: 2(2.5); Du: 1(2.49999) 0s
          2     2.1000000000e+01 Pr: 0(0) 0s
Model   status      : Optimal
Simplex   iterations: 2
Objective value     :  2.1000000000e+01
HiGHS run time      :          0.00


Row,name,value,rhs,slack,shadow_price,allowed_decrease,allowed_increase
,String,Float64,Float64,Float64,Float64,Float64,Float64
1,c1,6.0,6.0,0.0,0.5,-2.0,0.666667
2,c2,24.0,24.0,0.0,0.75,-4.0,12.0
